### 1. Import libraries

In [1]:
from AhpAnpLib import inputs_AHPLib as input
from AhpAnpLib import structs_AHPLib as str
from AhpAnpLib import calcs_AHPLib as calc
from AhpAnpLib import ratings_AHPLib as rate

### 2. Create model

In [2]:
# create the model using Model command, we can give the model a name here or leave it out
EmployEvaluation = str.Model("Employee Evaluation")

### 3. Create clusters, nodes and add nodes to clusters

In [3]:
# we create cluster, nodes of the model and we add nodes to their coresponding clusters using the add node2cluster command
# the model has three clusters
Goal = str.Cluster("Goal",1)
Criteria = str.Cluster("Criteria",2)
subCriteria = str.Cluster("Work Subcriteria",3)

# we then create nodes in each cluster
GoalNode = str.Node("GoalNode",1)
Goal.addNode2Cluster(GoalNode)

Dependability = str.Node("Dependability",1)
Education = str.Node("Education",2)
Experience = str.Node("Experience",3)
Attitude = str.Node("Attitude",4)
Work = str.Node("Work",5)
Leadership = str.Node("Leadership",6)
# we can use addMultipleNodes2Cluster command to add multiple nodes to the cluster at once
Criteria.addMultipleNodes2Cluster(Dependability,Education,Experience,Attitude,Work,Leadership)

# two subcriteria for Work
Quantity = str.Node("Quantity",1)
Quality = str.Node("Quality",2)
subCriteria.addMultipleNodes2Cluster(Quality,Quantity)

### 4. Add clustes to the model and set up connections

In [4]:
# we created goal, criteria, subcriteria above
# node we add them to the model and add connections to them then we get the AHP structure
EmployEvaluation.addCluster2Model(Goal)
EmployEvaluation.addMultipleClusters2Model(Criteria,subCriteria)
# when add connections we use cluster or node name
# to use the name we can either use the object.name to get the name 
# or we can specify the node/cluster name directly 
EmployEvaluation.addNodeConnectionFromNodeToAllNodesOfCluster(GoalNode.name,Criteria.name)
EmployEvaluation.addNodeConnectionFromNodeToAllNodesOfCluster("Work","Work Subcriteria")

In [5]:
# when use printStruct command to display and valide the model structure
EmployEvaluation.printStruct()

_________________________MODEL STRUCTURE_________________________
Name: Employee Evaluation
Type: pairwise

____________________________NODES_______________________________
GoalNode order: 1
Dependability order: 1
Education order: 2
Experience order: 3
Attitude order: 4
Work order: 5
Leadership order: 6
Quantity order: 1
Quality order: 2
____________________________CLUSTERS____________________________
Goal order: 1
Criteria order: 2
Work Subcriteria order: 3
_________________________NODE CONNECTIONS___________________________
Connections from node GoalNode order: 1
 to: Dependability order: 1
 to: Education order: 2
 to: Experience order: 3
 to: Attitude order: 4
 to: Work order: 5
 to: Leadership order: 6
No connections from Dependability
No connections from Education
No connections from Experience
No connections from Attitude
Connections from node Work order: 5
 to: Quality order: 2
 to: Quantity order: 1
No connections from Leadership
No connections from Quantity
No connections from

### 5.Set model as ratings model

In [6]:
# when create a model, the model by default is a pairewise compare model
# we use setModelTypeRatings command to change the model to a ratings model
EmployEvaluation.setModelTypeRatings()

### 6. Add bottom level criteria to use in the ratings model

In [7]:
# to set up the ratings model, we add Criteria first
# when add criteria model we add the bottom level criteria in the AHP model as criteria
# we can use addCriteriaByVar command by specifying criteria object name
# or we can use addCriteriaByName command by specifying criteria name
EmployEvaluation.rateModel.addCriteriaByVar(Dependability,Education,Experience,Attitude,Leadership)
EmployEvaluation.rateModel.addCriteriaByName("Quantity","Quality")

### 7. Add alternatives in the ratings model

In [8]:
# We can create alternatives in advance 
# or add new alternatives when adding alternatives in the ratings model
# here we add new alternatives directly to the ratings model
# when the alternatives are not created, we use the addAlternativesByName and provide alternative names
# then the alternatives will be created automatically
EmployEvaluation.rateModel.addAlternativesByName("Jim Kendall","Sally Brown","John Carter","Mi Sung","Arturo Chavez")

### 8. Create scales add the scales to the ratings model

In [9]:
# 1. Create a new scale (make pairewise comparisons in Excel questionnaire)
scale1=rate.RatScale("OutstandingtoUnsatisfactory")
scale1.defineScaleByValue(None,False,"Outstanding","Very Good","Good","Below Average","Unsatisfactory")
# add the scale to the model
EmployEvaluation.rateModel.addScaleByVar(scale1)
# 2. define scale by value 
scale2=rate.RatScale("EducationScale")
scale2.defineScaleByValue(None,False,["Doctorate",1],["Masters",0.433],["Bachelors",0.177],["High School",0.089])
EmployEvaluation.rateModel.addScaleByVar(scale2)
scale3=rate.RatScale("ExperienceScale")
scale3.defineScaleByValue(None,False,["More than 15 Years",1],["6 up to 15 years",0.428],["3 up to 6 years",0.168],["1 up to 3 years",0.07])
EmployEvaluation.rateModel.addScaleByVar(scale3)
scale4=rate.RatScale("AttitudeScale")
scale4.defineScaleByValue(None,False,["Enthused",1],["Above Average",0.359],["Average",0.153],["Negative",0.057])
EmployEvaluation.rateModel.addScaleByVar(scale4)
# 3. import from rcp file
scale5 = input.readRatScaleRCPfile("QuantityScale", "quantity.rcp")
scale6 = input.readRatScaleRCPfile("QualityScale", "quality.rcp")
EmployEvaluation.rateModel.addScaleByVar(scale5)
EmployEvaluation.rateModel.addScaleByVar(scale6)

### 9. Assign scales to Criteria

In [10]:
# we assign each criterion a scale 
# and the first parameter is the criterion name and the second parameter is scale name
EmployEvaluation.rateModel.assignScale2CriterionByName("Dependability",scale1.name)
EmployEvaluation.rateModel.assignScale2CriterionByName("Education","EducationScale")
EmployEvaluation.rateModel.assignScale2CriterionByName("Experience","ExperienceScale")
EmployEvaluation.rateModel.assignScale2CriterionByName("Attitude","AttitudeScale")
EmployEvaluation.rateModel.assignScale2CriterionByName("Leadership","OutstandingtoUnsatisfactory")
EmployEvaluation.rateModel.assignScale2CriterionByName("Quality","QualityScale")
EmployEvaluation.rateModel.assignScale2CriterionByName("Quantity","QuantityScale")

### 10. Export questionnair to Excel file and import filledin Excel file

In [11]:
# The command will export formated excel criteria pairwise compare file
input.export4ExcelQuestFull(EmployEvaluation,"Ratings_Employee_Eval_empty.xlsx",True)

In [12]:
# import judgment from Excel and then calculate criterion priorities then the results will be exported to an excel
calc.calcAHPMatricesSave2File(EmployEvaluation,"Ratings_Employee_Eval_withjudgments.xlsx","Ratings_Employee_Eval_criteriapriorities.xlsx",True,False,True)

In [13]:
# the command will export ratings scales pairwise comparison file in sheet 1
# and the ratings table will be export to the same Excel file in sheet 2
# we can then pairwise compare scale caterogies to get priorities of scales when needed
input.export4ExcelRatingsSetup(EmployEvaluation,"Ratings_Employee_Eval_ratingstable.xlsx",True)


load
No ratings tables available to load


In [15]:
# in this last step, we calculate ratings results and export it to an excel file
input.calcExcelRatings(EmployEvaluation,"Ratings_Employee_Eval_ratingstable_filled.xlsx","Ratings_Employee_Eval_ratingsresults.xlsx")

OutstandingtoUnsatisfactory [[1.000 1.742 3.049 8.000 16.667]
 [0.574 1.000 1.750 4.592 9.567]
 [0.328 0.571 1.000 2.624 5.467]
 [0.125 0.218 0.381 1.000 2.083]
 [0.060 0.105 0.183 0.480 1.000]]
[0.0]
EducationScale [[1.000 2.309 5.650 11.236]
 [0.433 1.000 2.446 4.865]
 [0.177 0.409 1.000 1.989]
 [0.089 0.206 0.503 1.000]]
[0.0, 0.0]
ExperienceScale [[1.000 2.336 5.952 14.286]
 [0.428 1.000 2.548 6.114]
 [0.168 0.393 1.000 2.400]
 [0.070 0.164 0.417 1.000]]
[0.0, 0.0, 0.0]
AttitudeScale [[1.000 2.786 6.536 17.544]
 [0.359 1.000 2.346 6.298]
 [0.153 0.426 1.000 2.684]
 [0.057 0.159 0.373 1.000]]
[0.0, 0.0, 0.0, 0.0]
OutstandingtoUnsatisfactory_cp [[1.000 2.203 4.405 14.493 23.256]
 [0.454 1.000 2.000 6.580 10.558]
 [0.227 0.500 1.000 3.290 5.279]
 [0.069 0.152 0.304 1.000 1.605]
 [0.043 0.095 0.189 0.623 1.000]]
[0.0, 0.0, 0.0, 0.0, 0.0]
QuantityScale [[1.000 1.832 2.740 7.812]
 [0.546 1.000 1.496 4.266]
 [0.365 0.668 1.000 2.852]
 [0.128 0.234 0.351 1.000]]
[0.0, 0.0, 0.0, 0.0, 0.0, 0